# main.ipynb

Compute neurophysiological differentiation from dF/F traces and decode stimuli

In [1]:
import multiprocessing
from itertools import product
from pathlib import Path

import numpy as np
import pandas as pd
import scipy
import scipy.spatial
import sklearn
import yaml
from joblib import Parallel, delayed
from sklearn.decomposition import PCA
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
from sklearn.model_selection import (StratifiedKFold, cross_val_score,
                                     cross_validate)
from sklearn.pipeline import Pipeline
from sklearn.utils import parallel_backend
from tqdm.auto import tqdm
from umap import UMAP

import metadata
from spectral_differentiation import compute_dff_differentiation, TRUNCATION_TOLERANCE
from analysis import (BLOCK_STIMULI, CONTINUOUS_NATURAL, SESSIONS,
                      UNSCRAMBLED_SCRAMBLED, get_cells)
from load import load_preprocessed_data
from metadata import METADATA, STIMULUS_METADATA

In [2]:
# Register tqdm with pandas for `progress_apply`
tqdm.pandas()

In [3]:
N_CPUS = multiprocessing.cpu_count()

## Load data

In [4]:
loaded_data = Parallel(n_jobs=min(len(SESSIONS), N_CPUS), verbose=5)(
    delayed(load_preprocessed_data)(session) for session in SESSIONS
)

[Parallel(n_jobs=44)]: Using backend LokyBackend with 44 concurrent workers.
[Parallel(n_jobs=44)]: Done   2 out of  44 | elapsed:    6.5s remaining:  2.3min
[Parallel(n_jobs=44)]: Done  11 out of  44 | elapsed:    9.1s remaining:   27.3s
[Parallel(n_jobs=44)]: Done  20 out of  44 | elapsed:   11.6s remaining:   13.9s
[Parallel(n_jobs=44)]: Done  29 out of  44 | elapsed:   14.8s remaining:    7.6s
[Parallel(n_jobs=44)]: Done  38 out of  44 | elapsed:   19.2s remaining:    3.0s
[Parallel(n_jobs=44)]: Done  44 out of  44 | elapsed:   23.4s finished


In [5]:
DFF = dict()
DATA = dict()
for session, (dff, data) in zip(SESSIONS, loaded_data):
    DFF[session] = dff
    DATA[session] = data

In [6]:
# Attach label columns to dF/F dataframes
for session, dff in tqdm(DFF.items()):
    DFF[session] = dff.merge(
        DATA[session].loc[:, ["stimulus", "trial", "stimulus_is_scrambled_pair", "stimulus_is_block"]],
        how="left",
        left_index=True,
        right_index=True,
    )

  0%|          | 0/44 [00:00<?, ?it/s]

# Spectral differentiation

### Build parameter space

Combinations of parameters to use for the sensitivity analysis.

In [7]:
args = (
    pd.DataFrame({
        # The length of the 'neurophysiological state' of the population, in seconds
        'state_length': [0.2, 0.5, 1, 2, 5, 10],
    })
    .merge(pd.DataFrame({
        # The metric with which to measure distances between neurophysiological states
        'metric': ["cityblock", "euclidean", "chebyshev"],
    }), how='cross')
    .merge(pd.DataFrame({
        # Whether to use log-spaced frequency bins in the spectral estimation
        'log_frequency': [False, True],
    }), how='cross')
    .merge(pd.DataFrame({
        # Which window to use
        'window': [
            None,
            'tukey',
            'kaiser',
        ],
        # The window parameter
        'window_param': [
            None,
            1/4,
            14,
        ],
        # The fraction of window overlap
        'overlap': [
            0,
            1/8,
            1/2
        ],
    }), how='cross')
)
args.head()

,state_length,metric,log_frequency,window,window_param,overlap
0,0.2,cityblock,False,None,NaN,0.000
1,0.2,cityblock,False,tukey,0.25,0.125
2,0.2,cityblock,False,kaiser,14.00,0.500
3,0.2,cityblock,True,None,NaN,0.000
4,0.2,cityblock,True,tukey,0.25,0.125


In [8]:
args = list(product(
    DFF.items(),
    (row for _, row in args.iterrows())
))

### Compute

In [9]:
def worker(session, dff, params):
    params = params.to_dict()
    # Compute
    result = compute_dff_differentiation(
        dff,
        fs=metadata.TWOP_SAMPLE_RATE,
        **params,
    )
    # Convert Series result to a DataFrame
    result.name = "differentiation"
    result = (
        result
        .reset_index(drop=False)
        # Record parameters & session
        .assign(
            **params,
            session=session,
        )
    )
    return result


results = Parallel(n_jobs=min(len(args), N_CPUS), verbose=5)(
    delayed(worker)(session, dff, params)
    for (session, dff), params in args
)
results = (
    # Put results into a DataFrame
    pd.concat(results)
    .reset_index(drop=True)
    # Add stimulus metadata
    .merge(STIMULUS_METADATA, on="stimulus")
)
# Only consider block stimuli
results = results.loc[results["stimulus_is_block"]]
print("Done.")

print(results.shape)
results.head()

[Parallel(n_jobs=160)]: Using backend LokyBackend with 160 concurrent workers.
[Parallel(n_jobs=160)]: Done 130 tasks      | elapsed:   24.7s
[Parallel(n_jobs=160)]: Done 328 tasks      | elapsed:   30.3s
[Parallel(n_jobs=160)]: Done 562 tasks      | elapsed:   35.1s
[Parallel(n_jobs=160)]: Done 832 tasks      | elapsed:   43.3s
[Parallel(n_jobs=160)]: Done 1138 tasks      | elapsed:   52.6s
[Parallel(n_jobs=160)]: Done 1480 tasks      | elapsed:  1.1min
[Parallel(n_jobs=160)]: Done 1858 tasks      | elapsed:  1.4min
[Parallel(n_jobs=160)]: Done 2272 tasks      | elapsed:  1.5min
[Parallel(n_jobs=160)]: Done 2722 tasks      | elapsed:  1.8min
[Parallel(n_jobs=160)]: Done 3208 tasks      | elapsed:  2.1min
[Parallel(n_jobs=160)]: Done 3730 tasks      | elapsed:  2.2min
[Parallel(n_jobs=160)]: Done 4288 tasks      | elapsed:  2.5min
[Parallel(n_jobs=160)]: Done 4752 out of 4752 | elapsed:  2.6min finished


Done.
(570240, 16)


,stimulus,trial,differentiation,state_length,metric,log_frequency,window,window_param,overlap,session,stimulus_type,stimulus_is_scrambled_pair,stimulus_is_block,stimulus_is_continuous,stimulus_code,stimulus_filename
0,conspecifics,0.0,17.202698,0.2,cityblock,False,None,NaN,0.0,717208879,natural,False,True,True,4,04-multiple-mice-deep.mp4.npy
1,conspecifics,1.0,15.736950,0.2,cityblock,False,None,NaN,0.0,717208879,natural,False,True,True,4,04-multiple-mice-deep.mp4.npy
2,conspecifics,2.0,16.225434,0.2,cityblock,False,None,NaN,0.0,717208879,natural,False,True,True,4,04-multiple-mice-deep.mp4.npy
3,conspecifics,3.0,20.011245,0.2,cityblock,False,None,NaN,0.0,717208879,natural,False,True,True,4,04-multiple-mice-deep.mp4.npy
4,conspecifics,4.0,12.592473,0.2,cityblock,False,None,NaN,0.0,717208879,natural,False,True,True,4,04-multiple-mice-deep.mp4.npy


### Attach metadata

In [10]:
metadata_subset = (
    METADATA.loc[METADATA.index.isin(SESSIONS)]
    .copy()
    .drop(
        columns=[
            "operator",
            "qc",
            "notes",
            "all_ophys_experiments",
            "all_ophys_sessions",
            "valid",
        ]
    )
)

df = results.merge(metadata_subset, on="session")

print(df.shape)
df.head()

(570240, 26)


,stimulus,trial,differentiation,state_length,metric,log_frequency,window,window_param,overlap,session,...,cre,area,depth,mouse_id,layer,ncells,start_time,order,sex,date_of_birth
0,conspecifics,0.0,17.202698,0.2,cityblock,False,None,NaN,0.0,717208879,...,Cux2,LM,175,389014,L2/3,41,2018-07-05 08:29:08.784,0,M,2018-03-18
1,conspecifics,1.0,15.736950,0.2,cityblock,False,None,NaN,0.0,717208879,...,Cux2,LM,175,389014,L2/3,41,2018-07-05 08:29:08.784,0,M,2018-03-18
2,conspecifics,2.0,16.225434,0.2,cityblock,False,None,NaN,0.0,717208879,...,Cux2,LM,175,389014,L2/3,41,2018-07-05 08:29:08.784,0,M,2018-03-18
3,conspecifics,3.0,20.011245,0.2,cityblock,False,None,NaN,0.0,717208879,...,Cux2,LM,175,389014,L2/3,41,2018-07-05 08:29:08.784,0,M,2018-03-18
4,conspecifics,4.0,12.592473,0.2,cityblock,False,None,NaN,0.0,717208879,...,Cux2,LM,175,389014,L2/3,41,2018-07-05 08:29:08.784,0,M,2018-03-18


### Compute normalizations and transforms

In [11]:
df['normalized differentiation'] = df['differentiation'] / np.sqrt(df['ncells'])
df['log(normalized differentiation)'] = np.log10(df['normalized differentiation'])

### Write results to disk

All parameters

In [12]:
df.to_parquet('results/sensitivity_analysis.parquet')
df.shape

(570240, 28)

Parameters used in main analysis

In [13]:
main = df.loc[
    (df["state_length"] == 1.0)
    & (df["metric"] == "euclidean")
    & df["window"].isna()
    & df["window_param"].isna()
    & (df["overlap"] == 0)
    & (df["log_frequency"] == False)
]
print(main.shape)

main.to_parquet('results/main.parquet')

(5280, 28)


# Multivariate differentiation

## Dimensionality reduction

In [14]:
with open('umap-params.yml', mode='rt') as f:
    UMAP_PARAMS = yaml.load(f, Loader=yaml.SafeLoader)

In [15]:
# Unscrambled & scrambled stimuli
args_list = [
    (session, dff.loc[dff['stimulus_is_scrambled_pair'] == True, :])
    for session, dff in tqdm(DFF.items())
]

  0%|          | 0/44 [00:00<?, ?it/s]

In [16]:
%%time

def worker(session, dff):
    labels = dff.loc[:, ['stimulus', 'trial']]
    signal = dff.loc[:, get_cells(dff)]
    
    transform = UMAP(**UMAP_PARAMS).fit_transform(signal)
    
    return pd.DataFrame(
        data=transform,
        index=dff.index,
        columns=[f'umap_{i}' for i in range(transform.shape[1])],
    ).join(labels)


results = Parallel(n_jobs=min(len(args_list), N_CPUS), verbose=5)(
    delayed(worker)(*args) for args in args_list
)

[Parallel(n_jobs=44)]: Using backend LokyBackend with 44 concurrent workers.
[Parallel(n_jobs=44)]: Done   2 out of  44 | elapsed:  1.6min remaining: 32.7min
[Parallel(n_jobs=44)]: Done  11 out of  44 | elapsed:  1.6min remaining:  4.8min
[Parallel(n_jobs=44)]: Done  20 out of  44 | elapsed:  1.6min remaining:  2.0min
[Parallel(n_jobs=44)]: Done  29 out of  44 | elapsed:  1.7min remaining:   52.2s
[Parallel(n_jobs=44)]: Done  38 out of  44 | elapsed:  1.7min remaining:   16.4s
[Parallel(n_jobs=44)]: Done  44 out of  44 | elapsed:  1.8min finished


CPU times: user 1.47 s, sys: 6.5 s, total: 7.97 s
Wall time: 1min 50s


In [17]:
transforms = pd.concat(results)

## Mean centroid distance

In [18]:
def centroid(points):
    return points.mean(axis=0).reshape(1, -1)


def centroid_distance(points):
    points = points.to_numpy()
    if points.ndim != 2:
        raise ValueError("points must be 2D")
    return scipy.spatial.distance.cdist(points, centroid(points), metric="euclidean")


def mean_centroid_distance(points):
    return np.mean(centroid_distance(points))

In [19]:
result = pd.Series(
    transforms.groupby(['session', 'stimulus', 'trial']).progress_apply(mean_centroid_distance),
    name='mean_centroid_distance',
).reset_index()

result['log(mean_centroid_distance)'] = np.log10(result['mean_centroid_distance'])

result = (
    result
    .merge(STIMULUS_METADATA, left_on='stimulus', right_index=True)
    .merge(METADATA, on='session')
)

  0%|          | 0/2200 [00:00<?, ?it/s]

In [20]:
result.to_parquet('results/mean_centroid_distance.parquet')

# Decoding

## Helper functions

In [21]:
def flatten_trial(data, target_length):
    """Truncate so each trial has the same number of frames."""
    if not abs(len(data) - target_length) <= TRUNCATION_TOLERANCE:
        raise ValueError(
            f"Expected length within tolerance {tolerance}; got {len(data)}"
        )
    return (
        data.iloc[:target_length]
        .drop(axis="columns", labels=["trial", "stimulus"])
        .to_numpy()
        .flatten()
    )


def get_labelled_data(session, stimuli, get_class):
    """Return data and classes.

    Reshapes and concatenates data for all stimuli.
    """
    dff = DFF[session]
    data = DATA[session]
    # Select stimulus subset and relevant columns
    dff = dff.loc[
        dff["stimulus"].isin(stimuli),
        get_cells(dff) + ['stimulus', 'trial']
    ]
    # X must be of shape (n_samples, n_features):
    # - samples are trials;
    # - features are the dF/F trace value for each cell for every sample within the trial,
    #   i.e. there are (30 s * 30 Hz * n_cells) features.
    # We need to truncate each trial to the minimum number of frames among all of them so
    # they each have the same number of features.
    target_length = int(dff.groupby(["trial", "stimulus"]).agg(len).min().min())
    X = (
        dff
        .groupby(["trial", "stimulus"])
        .apply(flatten_trial, target_length=target_length)
    )
    stimuli = X.index.get_level_values("stimulus")
    classes = stimuli.map(get_class)
    trials = X.index.get_level_values("trial")
    # Reshape to the (n_samples, n_features)
    samples = np.stack(X.to_numpy())
    return samples, classes.to_numpy(), trials.to_numpy(), stimuli.to_numpy()

## Compute

In [22]:
N_JOBS = -1
N_SPLITS = 5
CV = StratifiedKFold(n_splits=N_SPLITS)
PIPELINE = Pipeline(
    [
        ("reduce_dimensionality", PCA(n_components=0.99)),
        ("classify", LinearDiscriminantAnalysis(solver="lsqr", shrinkage="auto")),
    ]
)

In [23]:
identity = lambda x: x
category = STIMULUS_METADATA.stimulus_type.get

In [24]:
%%time

def worker(session, stimulus_set, get_class, scoring):
    X, y, trials, stimuli = get_labelled_data(
        session,
        stimuli=stimulus_set,
        get_class=get_class,
    )
    if scoring == "balanced_accuracy":
        scores = cross_val_score(PIPELINE, X, y, cv=CV, scoring=scoring, n_jobs=1)
        return pd.DataFrame([{scoring: scores.mean(), 'session': session}])
    if scoring == "f1_score":
        labels = sorted(np.unique(y))
        scoring = {
            label: sklearn.metrics.make_scorer(
                sklearn.metrics.f1_score,
                labels=[label],
                average="macro",
            )
            for label in labels
        }
        scores = cross_validate(PIPELINE, X, y, cv=CV, scoring=scoring, n_jobs=1)
        scores["session"] = session
        return pd.DataFrame(scores)
    raise ValueError("invalid scoring method")


for stimulus_set, scoring, get_class in tqdm(
    [
        # Figure 6
        (UNSCRAMBLED_SCRAMBLED, "balanced_accuracy", category),
        # Figure S8
        (BLOCK_STIMULI, "balanced_accuracy", identity),
        # Figure S9
        (CONTINUOUS_NATURAL, "f1_score", identity),
    ]
):
    if get_class is identity:
        decode_target = "identity"
    if get_class is category:
        decode_target = "category"

    if stimulus_set == UNSCRAMBLED_SCRAMBLED:
        stimulus_set_name = "unscrambled_scrambled"
    if stimulus_set == BLOCK_STIMULI:
        stimulus_set_name = "all"
    if stimulus_set == CONTINUOUS_NATURAL:
        stimulus_set_name = "continuous_natural"

    output_path = f"results/decoding__stimuli-{stimulus_set_name}__decode-{decode_target}__scoring-{scoring}.parquet"
    
    chance = 1 / len(set(map(get_class, stimulus_set)))

    with parallel_backend("loky", n_jobs=N_JOBS):
        results = Parallel()(
            delayed(worker)(session, stimulus_set, get_class, scoring)
            for session in SESSIONS
        )

    result_df = pd.concat(results).merge(METADATA, on="session")
    result_df["chance"] = chance

    result_df.to_parquet(output_path)

  0%|          | 0/3 [00:00<?, ?it/s]

CPU times: user 2min 29s, sys: 1min 18s, total: 3min 47s
Wall time: 4min 36s
